In [ ]:
from openpyxl import load_workbook, Workbook
import os

def add_data_to_excel(excel_file, tab_name, new_data):
    """
    Fun to add data in the excel sheet
    
    1. Excel sheet will be created if not exist
    2. Tabs will be created as per Topic to store data seprately for each topic
    
    """
    # Check if the Excel file exists
    if not os.path.exists(excel_file):
        # If the Excel file doesn't exist, create a new one and write the data to it
        wb = Workbook()
        ws = wb.active
        ws.title = tab_name
        ws.append(list(new_data.keys()))  # Add column headers
        ws.append(list(new_data.values()))  # Append new data
        wb.save(excel_file)
    else:
        # If the Excel file exists, open it and append the data to the specified tab
        wb = load_workbook(excel_file)
        if tab_name in wb.sheetnames:
            ws = wb[tab_name]
            ws.append(list(new_data.values()))  # Append new data
        else:
            # If the tab doesn't exist, create the tab and append the data to it
            ws = wb.create_sheet(title=tab_name)
            ws.append(list(new_data.keys()))  # Add column headers
            ws.append(list(new_data.values()))  # Append new data
        wb.save(excel_file)

In [ ]:
from confluent_kafka import Consumer, KafkaError

# Kafka broker address
bootstrap_servers = 'localhost:9092'

# Kafka topics to consume from
topics = ['calendar_topic', 'reviews_topic', 'listings_topic']  # Replace with your actual Kafka topic names

# Create Kafka Consumer
consumer_conf = {'bootstrap.servers': bootstrap_servers, 'group.id': 'my_consumer_group', 'auto.offset.reset': 'earliest'}
consumer = Consumer(consumer_conf)

# Subscribe to the topics
consumer.subscribe(topics)

output_file = "data.xlsx"

# Function to consume messages from Kafka
def consume_from_kafka():
    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition, consumer reached end of Kafka topic
                break
            else:
                print(msg.error())
                continue
        print(f"Data consumed for Topic: {msg.topic()}")

        # Fun to add data to the excel
        add_data_to_excel(
            excel_file=output_file, 
            tab_name=msg.topic(),
            new_data=eval(msg.value().decode('utf-8'))
        )

# Consume messages from Kafka
consume_from_kafka()

# Close Kafka Consumer
consumer.close()